Interacting Proteins

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import sys
sys.path.append('C:\\Users\\brittany henderson\\GitHub\\WhenMutationsDontMatter\\')
import plot_utils as p

In [2]:
gbm = cptac.Gbm()
endo = cptac.Endometrial()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


In [3]:
gene = 'PTEN'

In [4]:
mut_type_gbm = gbm.get_genotype_all_vars(gene)

prot = gbm.get_proteomics()
prot_list = list(prot.columns)

# merge cnv with genotype all mut type
prot_and_mutations = gbm.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()
del_wt

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 8)


Name,A1BG_proteomics,A2M_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00104,0.077630,0.487228,-0.254208,-0.144373,0.551881,-0.025276,-0.467451,-0.089511,-0.078806,0.329603,...,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990,Deletion
C3L-00365,-0.145975,0.798796,0.184242,-0.470603,NaN,0.390211,0.245466,-0.609998,0.118625,-0.086927,...,0.161975,-0.213093,0.235571,NaN,0.107421,0.048724,0.138403,-0.290141,0.405037,Deletion
C3L-00674,0.821991,1.096470,-0.094421,-0.106304,0.084578,0.176402,-0.248151,0.014061,-0.699773,-0.638462,...,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581,Deletion
C3L-00677,-0.064567,0.129385,0.047751,-0.118187,0.237434,NaN,0.303847,0.322163,-0.555479,-0.363414,...,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715,Deletion
C3L-01040,-0.763691,-1.031834,-0.217194,-0.695701,0.184173,-0.474816,-0.051789,0.344842,-0.642746,0.068863,...,-0.092502,0.010639,-0.465079,NaN,-0.500083,0.112651,1.004660,-0.230304,-0.102416,Deletion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-03183,0.230843,0.426624,0.420155,0.303960,0.078341,0.833128,0.205727,-0.434020,0.151202,0.208972,...,-0.021466,-0.049569,0.128461,-0.437797,-0.144343,-0.024981,-0.275897,-0.116367,0.351479,Deletion
C3N-03184,-0.635316,-0.478657,0.647004,-0.050322,-0.440517,0.092256,-0.006924,-0.511331,0.101877,0.128163,...,-0.034616,0.158269,NaN,NaN,0.906358,0.287317,0.745880,-0.320613,-0.180511,Deletion
C3N-03186,0.616640,0.767029,-0.231897,0.050841,0.304359,-0.384608,-0.205284,0.513575,0.151769,-0.307037,...,-0.110846,-0.069605,-0.183921,NaN,-0.681170,0.095601,-0.096716,0.049811,-0.409252,Deletion


In [5]:
cols = list(del_wt.columns[:-1])

g_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
g_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Comparison,P_Value
0,ARMH3_proteomics,5.396032e-11
1,CUTC_proteomics,1.593480e-10
2,PIP4K2A_proteomics,1.009419e-09
3,CUL2_proteomics,1.122076e-09
4,GDI2_proteomics,1.302273e-09
...,...,...
11136,ZNF805_proteomics,NaN
11137,ZNF813_proteomics,NaN
11138,ZNF814_proteomics,NaN
11139,ZNF888_proteomics,NaN


Endo - compare Nonsense and Frameshift mutations (not enough data for cnv del)

In [6]:
#Endo ttest
mut_type_en = endo.get_genotype_all_vars(gene)

prot = endo.get_proteomics()
prot_list = list(prot.columns)

# merge cnv with genotype all mut type
prot_and_mutations = endo.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_en = mut_type_en[['Mutation']] 
merged = ip_df.join(mut_type_en) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'] = np.where(
                del_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype', 'Truncation')
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 20 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 9)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Truncation    53
Wildtype      15
Name: Mutation, dtype: int64

In [7]:
cols = list(del_wt.columns[:-1])

e_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
e_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Comparison,P_Value
0,NOL10_proteomics,8.999599e-08
1,ABT1_proteomics,3.141336e-07
2,UTP25_proteomics,4.308134e-07
3,TOPBP1_proteomics,4.885533e-07
4,RIF1_proteomics,9.004982e-07
...,...,...
10994,ZNF586_proteomics,NaN
10995,ZNF630_proteomics,NaN
10996,ZNF772_proteomics,NaN
10997,ZNF79_proteomics,NaN


Ovarian

In [8]:
o = cptac.Ovarian()

In [9]:
mut_type_o = o.get_genotype_all_vars(gene)

prot = o.get_proteomics()
prot = o.reduce_multiindex(prot, levels_to_drop = 1)
prot_list = list(prot.columns)

# merge cnv with genotype all mut type
prot_and_mutations = o.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list)
prot_and_mutations = o.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_o = mut_type_o[['Mutation']] 
merged = ip_df.join(mut_type_o) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 98 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 01OV029, 02OV015 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: Due to dropping the specified levels, dataframe now has 1162 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 4)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 99 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packa

Wildtype_Tumor    46
Deletion          20
Name: Mutation, dtype: int64

In [10]:
cols = list(del_wt.columns[:-1])

o_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
o_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Comparison,P_Value
0,MMS19_proteomics,1.058914e-08
1,RACK1_proteomics,8.573522e-05
2,PI4K2A_proteomics,9.310383e-05
3,WAPL_proteomics,1.724680e-04
4,IDE_proteomics,2.011475e-04
...,...,...
10072,SAXO2_proteomics,NaN
10073,SH3GL3_proteomics,NaN
10074,SLC22A3_proteomics,NaN
10075,SSX2B_proteomics,NaN


Breast

In [11]:
b = cptac.Brca()

In [12]:
mut_type_b = b.get_genotype_all_vars(gene)

prot = b.get_proteomics()
prot = b.reduce_multiindex(prot, levels_to_drop = 1)
prot_list = list(prot.columns)

# merge cnv with genotype all mut type
prot_and_mutations = b.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list)
prot_and_mutations = b.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_b = mut_type_b[['Mutation']] 
merged = ip_df.join(mut_type_b) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 4)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 9)
cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 10)


Wildtype_Tumor    80
Deletion          24
Name: Mutation, dtype: int64

In [13]:
cols = list(del_wt.columns[:-1])

b_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
b_pval

,Comparison,P_Value
0,PTEN_proteomics,0.000004
1,EIF4H_proteomics,0.000025
2,MIEF1_proteomics,0.000031
3,TMSB10_proteomics,0.000043
4,DRG1_proteomics,0.000065
...,...,...
9766,POLR3E_proteomics,0.999356
9767,IGFBP7_proteomics,0.999595
9768,AAMP_proteomics,0.999613
9769,TRA2B_proteomics,0.999771


Colon

In [14]:
c = cptac.Colon()

In [15]:
mut_type_c = c.get_genotype_all_vars(gene)

prot = c.get_proteomics()
#prot = c.reduce_multiindex(prot, levels_to_drop = 1)
prot_list = list(prot.columns)

# merge cnv with genotype all mut type
prot_and_mutations = c.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_c = mut_type_c[['Mutation']] 
merged = ip_df.join(mut_type_c) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 100 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 201 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 9)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01CO001, 05CO005, 05CO007, 05CO014, 05

Wildtype_Tumor    76
Deletion          25
Name: Mutation, dtype: int64

In [16]:
cols = list(del_wt.columns[:-1])

c_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
c_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Comparison,P_Value
0,DFFA_proteomics,0.000037
1,WAPL_proteomics,0.000097
2,SEC14L2_proteomics,0.000136
3,GBF1_proteomics,0.000164
4,STK11IP_proteomics,0.000239
...,...,...
8062,ZNF841_proteomics,NaN
8063,ZNF888_proteomics,NaN
8064,ZNHIT6_proteomics,NaN
8065,ZNRD1_proteomics,NaN


Head and Neck

In [17]:
h = cptac.Hnscc()

In [18]:
mut_type_h = h.get_genotype_all_vars(gene)

prot = h.get_proteomics()
#prot = h.reduce_multiindex(prot, levels_to_drop = 1)
prot_list = list(prot.columns)

# merge cnv with genotype all mut type
prot_and_mutations = h.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_h = mut_type_h[['Mutation']] 
merged = ip_df.join(mut_type_h) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 109 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-02617, C3N-02727 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 382)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 176 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 9)


Wildtype_Tumor    87
Deletion          18
Name: Mutation, dtype: int64

In [19]:
cols = list(del_wt.columns[:-1])

h_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
h_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Comparison,P_Value
0,NLN_proteomics,1.113430e-09
1,AKR1C3_proteomics,1.032977e-08
2,AKR1C1_proteomics,6.931627e-08
3,ADI1_proteomics,1.601880e-07
4,EPHX1_proteomics,3.899394e-07
...,...,...
11739,ZNF664_proteomics,NaN
11740,ZNF836_proteomics,NaN
11741,ZNF862_proteomics,NaN
11742,ZSCAN31_proteomics,NaN


Combine

In [23]:
# merge significant pvals with outer join (keeps nonoverlapping) 
df = g_pval.merge(h_pval, on='Comparison', how='outer')#, rsuffix = '_ov')
df = df.rename(columns={'P_Value_x': 'Gbm', 'P_Value_y': 'Hnscc'})
df = df.merge(b_pval, on='Comparison', how='outer')
df = df.rename(columns={'P_Value': 'Brca'})
df = df.merge(o_pval, on='Comparison', how='outer')
df = df.rename(columns={'P_Value': 'Ovarian'})
df = df.merge(e_pval, on='Comparison', how='outer')
df = df.rename(columns={'P_Value': 'Endometrial'})
df = df.merge(c_pval, on='Comparison', how='outer') 
df = df.rename(columns={'P_Value': 'Colon'})

df = df.set_index('Comparison').transpose()
df

Comparison,ARMH3_proteomics,CUTC_proteomics,PIP4K2A_proteomics,CUL2_proteomics,GDI2_proteomics,ANXA7_proteomics,DCTN1_proteomics,PACS2_proteomics,DOCK5_proteomics,ACTR1A_proteomics,...,MAB21L2_proteomics,RETNLB_proteomics,TBX20_proteomics,TIFAB_proteomics,UGT2B15_proteomics,WTH3DI_proteomics,ZIC4_proteomics,ZNF419_proteomics,ZNF705E_proteomics,ZNF841_proteomics
Gbm,5.396032e-11,1.593480e-10,1.009419e-09,1.122076e-09,1.302273e-09,1.697911e-09,1.016640e-08,1.123526e-08,2.754160e-08,3.600370e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hnscc,5.584141e-04,1.894453e-04,1.543642e-03,4.005541e-01,6.309846e-01,6.956912e-01,1.775784e-03,2.179622e-01,2.897773e-01,5.794593e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brca,NaN,8.294666e-03,2.039519e-01,1.980426e-01,3.547123e-01,1.445817e-01,NaN,1.578679e-01,3.862962e-01,3.227128e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ovarian,NaN,6.016365e-02,3.891875e-01,7.861838e-01,5.694165e-01,9.500581e-01,2.593127e-01,4.862453e-01,2.015940e-01,3.202494e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Endometrial,1.112651e-03,8.286844e-02,2.017706e-01,8.643806e-01,6.165991e-02,1.901638e-01,2.745604e-04,5.402084e-01,3.687155e-03,2.061796e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Colon,NaN,4.115056e-01,1.998470e-01,3.490649e-01,9.905125e-01,5.759616e-03,3.005404e-01,6.696092e-02,6.834050e-02,4.430617e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.to_csv('del_vs_wt_pvals_all_proteins.csv')